In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('TL_Level_3a_NonPath_SP1_ZyMo_PSL_1_2_3_4_546.csv') 

# split data
X=data.iloc[:,51:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

     Schneider.Xr.A  Schneider.Xr.R  ...  Grantham.Xd.29  Grantham.Xd.30
0          0.084182        0.061733  ...        0.032505        0.032749
1          0.120118        0.083347  ...        0.031626        0.032946
2          0.129916        0.043305  ...        0.031596        0.031098
3          0.124517        0.045875  ...        0.031188        0.031233
4          0.144744        0.057897  ...        0.031939        0.030060
..              ...             ...  ...             ...             ...
541        0.136881        0.061596  ...        0.027010        0.029862
542        0.128324        0.064162  ...        0.031191        0.029338
543        0.059569        0.056259  ...        0.030661        0.031927
544        0.088450        0.071191  ...        0.032843        0.031953
545        0.095869        0.073311  ...        0.030617        0.029120

[546 rows x 100 columns]
0      1
1      1
2      1
3      1
4      1
      ..
541    4
542    4
543    4
544    4
545    4

In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.2,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(100, 4 - 1) = 3 components.
  ChangedBehaviorWarn

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(4 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 100)            40800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 128)            117248    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 4)                 260       
Total params: 207,716
Trainable params: 207,716
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
20/20 [==============================] - 37s 198ms/step - loss: 0.6809 - accuracy: 0.3868 - val_loss: 0.6156 - val_accuracy: 0.4114
Epoch 2/1000
20/20 [==============================] - 0s 25ms/step - loss: 0.5743 - accuracy: 0.4331 - val_loss: 0.4805 - val_accuracy: 0.4367
Epoch 3/1000
20/20 [==============================] - 0s 24ms/step - loss: 0.4654 - accuracy: 0.4644 - val_loss: 0.4490 - val_accuracy: 0.4873
Epoch 4/1000
20/20 [==============================] - 0s 24ms/step - loss: 0.4524 - accuracy: 0.5105 - val_loss: 0.4375 - val_accuracy: 0.5063
Epoch 5/1000
20/20 [==============================] - 0s 24ms/step - loss: 0.4396 - accuracy: 0.5239 - val_loss: 0.4363 - val_accuracy: 0.5127
Epoch 6/1000
20/20 [==============================] - 1s 26ms/step - loss: 0.4595 - accuracy: 0.4734 - val_loss: 0.4319 - val_accuracy: 0.5063
Epoch 7/1000
20/20 [==============================] - 0s 24ms/step - loss: 0.4270 - accuracy: 0.5042 - val_loss: 0.4207 - val_accuracy: 0.53

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.8920634920634921
test-acc = 0.7974683544303798
train-CFR =               precision    recall  f1-score   support

           0       0.94      0.95      0.94       151
           1       0.84      0.84      0.84       161
           2       0.97      0.90      0.93       165
           3       0.82      0.88      0.85       153

    accuracy                           0.89       630
   macro avg       0.89      0.89      0.89       630
weighted avg       0.89      0.89      0.89       630

test-CFR =               precision    recall  f1-score   support

           0       0.98      0.93      0.96        46
           1       0.69      0.75      0.72        36
           2       0.82      0.72      0.77        32
           3       0.70      0.75      0.73        44

    accuracy                           0.80       158
   macro avg       0.80      0.79      0.79       158
weighted avg       0.80      0.80      0.80       158

